# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gmaps_api_key

In [2]:
#aliasing key 
g_key = gmaps_api_key

In [3]:
#configuration
infile = '../WeatherPy/output/cities.csv'
# outfile = ''

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
df=pd.read_csv(infile, index_col=0)
df.head()

,city_id,city_name,cloudiness_pct,country_cd,date,humidity,city_lat,city_lng,max_temp,wind_speed,date_time
0,2194098,Ahipara,61,NZ,1621361484,77,-35.1667,173.1667,59.14,11.16,2021-05-18 18:11:24
1,2027296,Aykhal,100,RU,1621361484,97,66.0000,111.5000,25.03,7.65,2021-05-18 18:11:24
2,5312476,Safford,1,US,1621361484,17,32.8340,-109.7076,83.32,18.41,2021-05-18 18:11:24
3,2263905,Ribeira Brava,1,PT,1621361485,60,32.6500,-17.0667,75.70,3.98,2021-05-18 18:11:25
4,2155415,New Norfolk,86,AU,1621361485,76,-42.7826,147.0587,51.91,1.99,2021-05-18 18:11:25


591

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [93]:
#set locations and weight series. First remove nulls
df = df.dropna() #defaults to drop entire rows if any NaN is encountered, which is the behavior desired.

city_locations = df[['city_lat', 'city_lng']]
humidity  = df['humidity']

In [94]:
#Set up map and heat layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
# criteria: conjuct of: max temp on the open interval (70,80), wind speed <10MPH, zero cloudiness



df_comfy = df[(df['max_temp']>70) & (df['max_temp'] <80)&(df['wind_speed']<10)&(df['cloudiness_pct']==0)]
len(df_comfy)

6

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:
cols_in_hotel_df = ['city_name', 'country_cd', 'city_lat', 'city_lng', 'hotel_nm']

hotel_df = df_comfy[cols_in_hotel_df]
#hotel_df['hotel_nm'] = np.nan

In [44]:
# renaming column to have consistent naming convention (name is always abbreviated as 'nm')
# saves time by making it easier to remember column names to reference later
hotel_df = hotel_df.rename({'city_name':'city_nm'}, axis = 'columns')  #using inplace=True generated a warning, so using equation instead 
hotel_df.head()

,city_nm,country_cd,city_lat,city_lng,hotel_nm
126,Goderich,CA,43.7501,-81.7165,NaN
218,Saint-Pierre,RE,-21.3393,55.4781,NaN
300,Vila Velha,BR,-20.3297,-40.2925,NaN
309,Cap Malheureux,MU,-19.9842,57.6142,NaN
381,Murgab,TM,37.4966,61.9714,NaN


In [46]:
hotel_df = hotel_df.reset_index(drop=True)  #resetting index because I have a feeling this will be needed later.
hotel_df   # only have six rows so displaying the whole thing

,city_nm,country_cd,city_lat,city_lng,hotel_nm
0,Goderich,CA,43.7501,-81.7165,NaN
1,Saint-Pierre,RE,-21.3393,55.4781,NaN
2,Vila Velha,BR,-20.3297,-40.2925,NaN
3,Cap Malheureux,MU,-19.9842,57.6142,NaN
4,Murgab,TM,37.4966,61.9714,NaN
5,Kiryat Gat,IL,31.6100,34.7642,NaN


In [56]:

#constant params for this exercise
target_search = "Hotel"  
target_radius = 5000     
target_type = 'establishment'

In [57]:
#form list of dictionaries holding the parameters for each search
    #loop through coordinates to get list of strings
city_coords = []
for i in range(len(hotel_df)):
    target_coordinates =  ", ".join((str(hotel_df.city_lat[i]), str(hotel_df.city_lng[i])))  #this is probably not the most elegant approach, however I tend to make mistakes with string concatenation, so trying something new
    city_coords.append(target_coordinates)
    
    #set up master list
params_list = []    
    
for i in range(len(hotel_df)):
    target_coordinates = city_coords[i]
    #leverage constants
    one_dict = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    params_list.append(one_dict)
    

In [85]:
#output list initialize
found_hotels = []

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using the iteration over params list of dictionaries
for i in range(len(params_list)):
    response = requests.get(base_url, params=params_list[i])
    places_data = response.json()
    if places_data['status'] != 'OK':
        found_hotels.append("none found within range")
    else:
        found_hotels.append(places_data["results"][0]["name"])

    
#checking the output
print(found_hotels)

['Samuels Hotel', 'Villa Delisle, Hotel & Spa', 'Hotel Senac Ilha do Boi', 'Ventura Hotel', 'none found within range', 'Desert Gat']


In [86]:
hotel_df['hotel_nm'] = found_hotels

In [87]:
hotel_df

,city_nm,country_cd,city_lat,city_lng,hotel_nm
0,Goderich,CA,43.7501,-81.7165,Samuels Hotel
1,Saint-Pierre,RE,-21.3393,55.4781,"Villa Delisle, Hotel & Spa"
2,Vila Velha,BR,-20.3297,-40.2925,Hotel Senac Ilha do Boi
3,Cap Malheureux,MU,-19.9842,57.6142,Ventura Hotel
4,Murgab,TM,37.4966,61.9714,none found within range
5,Kiryat Gat,IL,31.6100,34.7642,Desert Gat


In [89]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_nm}</dd>
<dt>City</dt><dd>{city_nm}</dd>
<dt>Country</dt><dd>{country_cd}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["city_lat", "city_lng"]]

In [95]:
##create figure

fig = gmaps.figure()


# heat layer
heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)



# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))